In [16]:
#!pip install nltk
#!pip install pytorch_lightning
#!pip install gensim
#!pip install pyserini==0.12.0
#!pip install python-terrier
#!pip install ipywidgets
#!pip install --upgrade notebook jupyter
#!jupyter nbextension enable --py widgetsnbextension
#!pip install tensorboard tensorboardX
#!pip install torch-tb-profiler

  Obtaining dependency information for torch-tb-profiler from https://files.pythonhosted.org/packages/46/d0/891ec43349f287ea5c313ebc1320e4eda38ccd4c7a0951657213467eaab5/torch_tb_profiler-0.4.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.3 MB/s eta 0:00:00 0:00:01


In [1]:
# Import libraries
import os
import json
import random
import numpy as np
import pandas as pd
from tqdm import tqdm
import matplotlib.pyplot as plt

import torch
from torch.utils.data import DataLoader, random_split

import pytorch_lightning as pl
from pytorch_lightning import Trainer
from pytorch_lightning.loggers import TensorBoardLogger

import nltk
from gensim.models import Word2Vec

from src.dataset import mydataset
from src.utils import utils, dataset_utils, eval_utils, trie

from src.models import siamese, attention_siamese, siamese_triplet, s2stransformer, s2s_transf, s2s_transf_semantic

In [2]:
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/alirezarafiei/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/alirezarafiei/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/alirezarafiei/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [2]:
EMBEDDING_SIZE = 120
MAX_TOKENS = 7
K = 10      # K param metrics

QUERIES_PATH = './materials/queries.json'
DOCS_PATH = './materials/documents.json'
W2V_PATH = './materials/word2vec_model.bin'
BT_MOD_PATH = './materials/bt_model.pth'

In [3]:
# If the json files of documents does not exists
if not os.path.exists(QUERIES_PATH) or not os.path.exists(DOCS_PATH) or not os.path.exists(W2V_PATH):
    # Build dictionaries and corpus
    queries, documents, corpus = dataset_utils.build_dicts(
        max_topics=None, max_docs=10)
    # Write to file the dictionaries
    with open(QUERIES_PATH, 'w') as json_file:
        json.dump(queries, json_file)
    with open(DOCS_PATH, 'w') as json_file:
        json.dump(documents, json_file)

    # Train the word2vec model
    w2v_model = Word2Vec(
        sentences=corpus, vector_size=EMBEDDING_SIZE,
        window=MAX_TOKENS if MAX_TOKENS else 5,
        min_count=1, sg=0, epochs=10)
    
    # Save w2v model
    w2v_model.save(W2V_PATH)

# Read from json
with open(QUERIES_PATH, 'r') as json_file:
    queries = json.load(json_file)
with open(DOCS_PATH, 'r') as json_file:
    documents = json.load(json_file)

# Load model
w2v_model = Word2Vec.load(W2V_PATH)

In [4]:
# Compute w2v embeddings for queries
for id in tqdm(queries, "Embedding queries"):
    raw_query = queries[id]['raw']
    queries[id]['emb'] = dataset_utils.compute_embedding(
        raw_query, w2v_model, 1)
    queries[id]['first_L_emb'] = dataset_utils.compute_embedding(
        raw_query, w2v_model, MAX_TOKENS)

# Compute w2v embeddings for documents
for docid in tqdm(documents, "Embedding documents"):
    raw_doc = documents[docid]['raw']
    documents[docid]['emb'] = dataset_utils.compute_embedding(
        raw_doc, w2v_model)
    documents[docid]['first_L_emb'] = dataset_utils.compute_embedding(
        raw_doc, w2v_model, MAX_TOKENS)

Embedding documents: 100%|██████████| 67869/67869 [00:33<00:00, 2022.98it/s]


In [5]:
# Create datasets for siamese models
pairs_dataset = mydataset.QueryDocumentDataset(queries, documents)
triplets_dataset = mydataset.TripletQueryDocumentDataset(queries, documents)

Building QueryDocumentDataset:   0%|          | 0/6980 [00:00<?, ?it/s]

Building TripletQueryDocumentDataset:   0%|          | 0/6980 [00:00<?, ?it/s]

In [6]:
# Create datasets for seq2seq models
dd_dataset = mydataset.DocumentDataset(documents)
ret_dataset = mydataset.RetrievalDataset(documents, queries)

You are using the default legacy behaviour of the <class 'transformers.models.t5.tokenization_t5.T5Tokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface/transformers/pull/24565
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Building DocumentDataset:   0%|          | 0/67869 [00:00<?, ?it/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


Building RetrievalDataset:   0%|          | 0/6980 [00:00<?, ?it/s]

In [7]:
# QueryDocumentDataset sample print (by default, IDs will be printed)
print(
    f"(query, doc, relevance) = "
    f"({pairs_dataset[0][0]}, {pairs_dataset[0][1]}, {pairs_dataset[0][2]})"
)

# TripletQueryDocumentDataset sample print (by default, IDs will be printed)
print(
    f"(query, doc+, doc-) = "
    f"({triplets_dataset[0][0]}, {triplets_dataset[0][1]}, {triplets_dataset[0][2]})"
)

# DocumentDataset sample print
print("\ndoc_emb:", dd_dataset[0][0], "\ndocid_emb:", dd_dataset[0][1])
print("decoded docid:", dd_dataset.decode_docid(dd_dataset[0][1]))
print("decoded doc:", dd_dataset.tokenizer.decode(dd_dataset[0][0]))

# RetrievalDataset sample print
print("\nquery_emb:", ret_dataset[0][0], "\ndocid_emb:", ret_dataset[0][1])
print("decoded docid:", ret_dataset.decode_docid(ret_dataset[0][1]))
print("decoded query:", ret_dataset.tokenizer.decode(ret_dataset[0][0]))

(query, doc, relevance) = (1102330, 7867440, 1)
(query, doc+, doc-) = (1102330, 7867440, 848780)

doc_emb: tensor([ 5696,  8325,   819,  1836,  9657,  1406,  7178, 19511,   819,  1836,
           78,    60,  9657, 20743,  3841,   706,   228,     3,  9052,   226,
          159,    51,  3841, 20743,   158,    32,   102,    40, 20743,  3841,
         3841,     1]) 
docid_emb: tensor([12,  7,  8,  6,  7,  4,  4,  6, 10, 11, 11])
decoded docid: 7867446
decoded doc: suffer frequent headach jaw pain wake dull headach sore jaw grind teeth night could bruxism teeth grind peopl grind teeth teeth</s>

query_emb: tensor([  158,    32,   102,    40, 20743,  3841,  2085,     1,     0]) 
docid_emb: tensor([12,  7,  8,  6,  7,  4,  4,  6, 10, 11, 11])
decoded docid: 7867446
decoded query: peopl grind teeth sleep</s><pad>


In [21]:
%load_ext tensorboard

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [22]:
# Change dataset return type to 'emb' (w2v embeddings)
pairs_dataset.set_ret_type('emb')

# Initialize the siamese network
siamese_net = siamese.SiameseNetwork(
    input_size=EMBEDDING_SIZE,
    conv_channels=[8,16,32])

# Train the model
train_set_sn, valid_set_sn, metrics = utils.train_model(
    pairs_dataset, siamese_net, max_epochs=3,
    batch_size=1024, split_ratio=0.8, num_workers=2)

%tensorboard --logdir lightning_logs/

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Missing logger folder: /Users/alirezarafiei/IR_DSI_project/lightning_logs

  | Name            | Type       | Params
-----------------------------------------------
0 | siamese_network | Sequential | 2.0 K 
1 | fc_layers       | Sequential | 984 K 
2 | fc              | Linear     | 9     
3 | sigmoid         | Sigmoid    | 0     
4 | criterion       | BCELoss    | 0     
-----------------------------------------------
986 K     Trainable params
0         Non-trainable params
986 K     Total params
3.945     Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/alirezarafiei/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'val_dataloader' to speed up the dataloader worker initialization.


val_loss_avg:  tensor(0.7042, device='mps:0')
accuracy:  0.48486328125


/Users/alirezarafiei/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:436: Consider setting `persistent_workers=True` in 'train_dataloader' to speed up the dataloader worker initialization.


...

train_loss_avg:  tensor(0.6935, device='mps:0')
val_loss_avg:  tensor(0.6932, device='mps:0')
accuracy:  0.49805508128234316


Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=3` reached.


train_loss_avg:  tensor(0.6933, device='mps:0')
val_loss_avg:  tensor(0.6932, device='mps:0')
accuracy:  0.5014484311853137


ERROR: Failed to launch TensorBoard (exited with 1).
Contents of stderr:
I0427 16:55:25.226720 12280950784 plugin.py:429] Monitor runs begin
Address already in use
Port 6006 is in use by another program. Either identify and stop that program, or start the server with a different port.

In [19]:
# Set model to evaluation mode
siamese_net.eval()

# Extract first query
query = list(queries.keys())[0]
# Get the list of relevant docids for the query
docids_list = queries[query]['docids_list']

# Compute top-k docids
top_k_ids = eval_utils.top_k_docids_siamese(
    siamese_net, queries[query], documents, k=K)

# Print top-k docids and relevant docid
print(f"Top {K} ids:", top_k_ids)
print("Relevant ids:", docids_list)
print("Common elements", np.isin(top_k_ids, docids_list).sum())

Computing Top 10 DocIDs:   0%|          | 0/67869 [00:00<?, ?it/s]

Top 10 ids: ['1142358' '2742763' '6114991' '8745760' '6841142' '8034100' '7670442'
 '8261837' '7093300' '8261838']
Relevant ids: ['7867446', '3368049', '4778576', '2914344', '2045120', '7867439', '6110464', '2045118', '7867440', '8699805']
Common elements 0


In [20]:
# Change return type to 'id' to get query IDs
pairs_dataset.set_ret_type('id')
# Randomly select N triplets from validation set
selected_triplets = random.sample(list(valid_set_sn), 10)
# Extract queries from the selected triplets
random_queries = [triplet[0] for triplet in selected_triplets]

# Compute mean metrics
running_mean_AP, running_mean_PatK, running_mean_RatK = eval_utils.compute_Mean_metrics(
    siamese_net, random_queries, queries, documents, k=K, model_type='siamese')
# Print mean average precision
print("Mean Average Precision (MAP):", running_mean_AP)
# Print mean precision at k
print(f"Mean Precison@{K}", running_mean_PatK)
# Print mean recall at k
print(f"Mean Recall@1000", running_mean_RatK)

Computing Mean Metrics:   0%|          | 0/10 [00:00<?, ?it/s]

Computing Top 1000 DocIDs:   0%|          | 0/67869 [00:00<?, ?it/s]

Computing Top 1000 DocIDs:   0%|          | 0/67869 [00:00<?, ?it/s]

Computing Top 1000 DocIDs:   0%|          | 0/67869 [00:00<?, ?it/s]

Computing Top 1000 DocIDs:   0%|          | 0/67869 [00:00<?, ?it/s]

...

Computing Top 1000 DocIDs:   0%|          | 0/67869 [00:00<?, ?it/s]

Computing Top 1000 DocIDs:   0%|          | 0/67869 [00:00<?, ?it/s]

Computing Top 1000 DocIDs:   0%|          | 0/67869 [00:00<?, ?it/s]

Computing Top 1000 DocIDs:   0%|          | 0/67869 [00:00<?, ?it/s]

Mean Average Precision (MAP): 0.0
Mean Precison@10 0.0
Mean Recall@1000 0.0


In [ ]:
# Change return type to 'id' to get query IDs
pairs_dataset.set_ret_type('id')
# Randomly select N triplets from train set
selected_triplets = random.sample(list(train_set_sn), 10)
# Extract queries from the selected triplets
random_queries = [triplet[0] for triplet in selected_triplets]

# Compute mean metrics
running_mean_AP, running_mean_PatK, running_mean_RatK = eval_utils.compute_Mean_metrics(
    siamese_net, random_queries, queries, documents, k=K, model_type='siamese')
# Print mean average precision
print("Mean Average Precision (MAP):", running_mean_AP)
# Print mean precision at k
print(f"Mean Precison@{K}", running_mean_PatK)
# Print mean recall at k
print(f"Mean Recall@1000", running_mean_RatK)

In [10]:
# Change dataset return type to 'emb' (embeddings)
pairs_dataset.set_ret_type('first_L_emb')

# Initialize siamese transformer network
siamese_transformer = attention_siamese.SiameseTransformer(
    embedding_size=EMBEDDING_SIZE*MAX_TOKENS)

# Train the model
train_set_st, valid_set_st = utils.train_model(
    pairs_dataset, siamese_transformer,
    max_epochs=10, batch_size=1024, split_ratio=0.8)

GPU available: True (mps), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs

  | Name        | Type              | Params
--------------------------------------------------
0 | transformer | CustomTransformer | 3.4 M 
--------------------------------------------------
3.4 M     Trainable params
0         Non-trainable params
3.4 M     Total params
13.443    Total estimated model params size (MB)


Sanity Checking: |          | 0/? [00:00<?, ?it/s]

/Users/alirezarafiei/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'val_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


val_loss_avg:  tensor(0.7741, device='mps:0')
accuracy:  0.51416015625


/Users/alirezarafiei/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:441: The 'train_dataloader' does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` to `num_workers=13` in the `DataLoader` to improve performance.


...

train_loss_avg:  tensor(0.6083, device='mps:0')
val_loss_avg:  tensor(0.6147, device='mps:0')
accuracy:  0.692743895309312


Validation: |          | 0/? [00:00<?, ?it/s]

train_loss_avg:  tensor(0.6003, device='mps:0')
val_loss_avg:  tensor(0.6166, device='mps:0')
accuracy:  0.6908153891563416


Validation: |          | 0/? [00:00<?, ?it/s]

/Users/alirezarafiei/miniforge3/lib/python3.10/site-packages/pytorch_lightning/trainer/call.py:54: Detected KeyboardInterrupt, attempting graceful shutdown...


In [ ]:
# Set model to evaluation mode
siamese_transformer.eval()

# Extract first query
query = list(queries.keys())[0]
# Get the list of relevant docids for the query
docids_list = queries[query]['docids_list']

# Compute top-k docids
top_k_ids = eval_utils.top_k_docids_siamese(
    siamese_transformer, queries[query], documents, k=K)

# Print top-k docids and relevant docid
print(f"Top {K} ids:", top_k_ids)
print("Relevant ids:", docids_list)
print("Common elements", np.isin(top_k_ids, docids_list).sum())

  Obtaining dependency information for torch-tb-profiler from https://files.pythonhosted.org/packages/46/d0/891ec43349f287ea5c313ebc1320e4eda38ccd4c7a0951657213467eaab5/torch_tb_profiler-0.4.3-py3-none-any.whl.metadata
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 10.3 MB/s eta 0:00:00 0:00:01


In [ ]:
# Change return type to 'id' to get query IDs
pairs_dataset.set_ret_type('id')
# Randomly select N triplets from validation set
selected_triplets = random.sample(list(valid_set_st), 10)
# Extract queries from the selected triplets
random_queries = [triplet[0] for triplet in selected_triplets]

# Compute mean metrics
running_mean_AP, running_mean_PatK, running_mean_RatK = eval_utils.compute_Mean_metrics(
    siamese_transformer, random_queries, queries,
    documents, k=K, model_type='siamese')
# Print mean average precision
print("Mean Average Precision (MAP):", running_mean_AP)
# Print mean precision at k
print(f"Mean Precison@{K}", running_mean_PatK)
# Print mean recall at k
print(f"Mean Recall@1000", running_mean_RatK)

In [ ]:
# Change return type to 'id' to get query IDs
pairs_dataset.set_ret_type('id')
# Randomly select N triplets from validation set
selected_triplets = random.sample(list(train_set_st), 10)
# Extract queries from the selected triplets
random_queries = [triplet[0] for triplet in selected_triplets]

# Compute mean metrics
running_mean_AP, running_mean_PatK, running_mean_RatK = eval_utils.compute_Mean_metrics(
    siamese_transformer, random_queries, queries,
    documents, k=K, model_type='siamese')
# Print mean average precision
print("Mean Average Precision (MAP):", running_mean_AP)
# Print mean precision at k
print(f"Mean Precison@{K}", running_mean_PatK)
# Print mean recall at k
print(f"Mean Recall@1000", running_mean_RatK)

In [ ]:
# Change dataset return type to 'emb' (embeddings)
triplets_dataset.set_ret_type('emb')

# Initialize the Siamese Lightning module
siamese_lightning_module = siamese_triplet.SiameseNetworkPL(
    input_size=EMBEDDING_SIZE, output_size=64,
    margin=1.0, learning_rate=0.001, arch_type='linear')

# Train the model
train_set_sc, valid_set_sc = utils.train_model(
    triplets_dataset, siamese_lightning_module,
    max_epochs=10, batch_size=1024, split_ratio=0.8)

# Start TensorBoard within the notebook
%tensorboard --logdir lightning_logs/

### pyterrier-based approaches

In [5]:
import pyterrier as pt
if not pt.started():
    pt.init()
import string

PyTerrier 0.10.0 has loaded Terrier 5.8 (built by craigm on 2023-11-01 18:05) and terrier-helper 0.0.8

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [15]:
docs = pd.DataFrame([
    {"docno": doc_id, "text": details['raw']}
    for doc_id, details in documents.items()
])

indexer = pt.DFIndexer("./materials/index_pt/index", overwrite=True)
index_ref = indexer.index(docs["text"], docs["docno"])

quer = pd.DataFrame([{"qid": str(qid), "query": details['raw']} for qid, details in queries.items()])
qrels = pd.DataFrame([{"qid": str(qid), "docno": docno, "label": 1} for qid, details in queries.items() for docno in details['docids_list']])

translator = str.maketrans('', '', string.punctuation)
def remove_punctuation(text):
    text_no_punctuation = text.translate(translator)
    return text_no_punctuation

quer['query'] = quer['query'].apply(remove_punctuation)


In [16]:
# Load the index
index = pt.IndexFactory.of(index_ref)

# Create a BM25 retrieval model
bm25 = pt.BatchRetrieve(index, wmodel="BM25")

tf_idf = pt.BatchRetrieve(index, wmodel="TF_IDF")

pl2 = pt.BatchRetrieve(index, wmodel="PL2")

evaluation = pt.Experiment(
    [bm25, tf_idf, pl2],
    quer,
    qrels,
    eval_metrics=['map', 'ndcg', 'P', 'recall'],
    round=4,
    verbose="true"
)

print(evaluation)

pt.Experiment:   0%|          | 0/3 [00:00<?, ?system/s]

pt.Experiment: 100%|██████████| 3/3 [03:05<00:00, 61.91s/system]

         name     map    ndcg     P@5    P@10    P@15    P@20    P@30  P@100  \
0    BR(BM25)  0.9456  0.9751  0.9614  0.9167  0.6399  0.4853  0.3264  0.099   
1  BR(TF_IDF)  0.9458  0.9752  0.9617  0.9171  0.6400  0.4854  0.3264  0.099   
2     BR(PL2)  0.9378  0.9720  0.9552  0.9043  0.6368  0.4843  0.3261  0.099   

    P@200  ...  P@1000     R@5    R@10    R@15    R@20    R@30   R@100  \
0  0.0496  ...    0.01  0.4807  0.9167  0.9599  0.9706  0.9791  0.9896   
1  0.0496  ...    0.01  0.4808  0.9171  0.9601  0.9708  0.9792  0.9896   
2  0.0496  ...    0.01  0.4776  0.9043  0.9553  0.9686  0.9784  0.9896   

    R@200   R@500  R@1000  
0  0.9923  0.9944  0.9952  
1  0.9922  0.9944  0.9952  
2  0.9924  0.9944  0.9952  

[3 rows x 21 columns]
